In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

Using TensorFlow backend.


In [2]:
train_data = np.load('train_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])

test_data = np.load('test_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])

train = np.zeros([2000, 150, 150, 3])
for i in range(2000):
    train[i] = train_data[i]
    
test = np.zeros([800, 150, 150, 3])
for i in range(800):
    test[i] = test_data[i]

In [3]:
batch_size = 16

In [9]:
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1./255)
    
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')
    
    generator = datagen.flow(train, 
                             batch_size=batch_size,
                             shuffle=False)
    
    bottleneck_features_train = model.predict_generator(generator, 2000 // batch_size)
    np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)
    
    generator = datagen.flow(test, 
                             batch_size=batch_size, 
                             shuffle=False)
    
    bottleneck_features_validation = model.predict_generator(generator, 800 // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

In [5]:
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy'))
    train_labels = np.array([0] * 1000 + [1] * 1000)
    
    test_data = np.load(open('bottleneck_features_validation.npy'))
    test_labels = np.array([0] * 400 + [1] * 400)
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data[1:]))
    model.add(Dense(256), activation='relu')
    model.add(Dropout(0.5))
    model.add(Dense(1), activation='sigmoid')
    
    model.compile(loss='binary_crossentropy', 
                  optimizer='rmsprop', 
                  metrics=['accuracy'])
    
    model.fit(train_data,
              train_labels, 
              epoch=50,
              batch_size=batch_size,
              validation_data=(test_data, test_labels))
    model.save_weights('bottleneck_fc_model.h5')

In [ ]:
save_bottleneck_features()